In [218]:
import pandas as pd
import re
import numpy as np
import datetime
import calendar

In [219]:
df = pd.read_csv('test.csv')

In [220]:
pattern_age = re.compile(r"[0-9][0-9]세") 
pattern_age_num = re.compile(r"[0-9][0-9]") 
pattern_age_se = re.compile(r"[0-9][0-9]\s?(?=세)")
pattern_age_wave = re.compile(r"[0-9][0-9]\s?(?=~)")
pattern_age_more = re.compile(r"[0-9][0-9]\s?세?\s?이상")
pattern_age_less = re.compile(r"[0-9][0-9]\s?세?\s?이하")
pattern_age_less2 = re.compile(r"[0-9][0-9]\s?세?\s?미만")

In [221]:
age_col = '연령'

In [222]:
def remove_dup_list(x):
    temp_set = set()
    res = []
    for idx in x:
        if idx not in temp_set:
            res.append(idx)
            temp_set.add(idx)
    return res

In [223]:
def age_parse(df):
    
    df_res = df.copy()
    
    x = df_res[age_col]
    
    df_res['start_age'] = None
    df_res['end_age'] =None
    
    more = []
    less = []
    wave = []
    se = []
    less2=[]
    
    temp = []
    start = []
    end = []
    
    if pattern_age_more.search(x):
        more.extend(pattern_age_more.findall(x))
    
    if pattern_age_less.search(x):
        less.extend(pattern_age_less.findall(x))
    
    if pattern_age_wave.search(x):
        wave.extend(pattern_age_wave.findall(x))
    
    if pattern_age_se.search(x):
        se.extend(pattern_age_se.findall(x))
    
    if pattern_age_less2.search(x):
        less2.extend(pattern_age_less2.findall(x))
    
    
    for i in se:
        temp.extend(pattern_age_num.findall(i))
    for i in wave:
        start.extend(pattern_age_num.findall(i))
    for i in less:
        end.extend(pattern_age_num.findall(i))
    for i in more:
        start.extend(pattern_age_num.findall(i))
    for i in less2:
        end.extend(pattern_age_num.findall(i))
    
    start = remove_dup_list(start)
    temp = remove_dup_list(temp)
    
    temp2 = temp.copy()
    for kda in temp2:
        if kda in start:
            temp.remove(kda)
        elif kda in end:
            temp.remove(kda)
        
    if len(temp)>1:
        start.append(temp[0])
        end.append(temp[1])
    elif len(temp)==1:
        if temp[0] not in start and temp[0] not in end:
            if len(start) == 0:
                start.append(temp[0])
            elif len(end) == 0:
                end.append(temp[0])
    
    if len(start)>0:
        df_res['start_age'] = start[0]
    
    if len(end)>0:
        df_res['end_age'] = end[0]
    
    return df_res

In [224]:
df = df.apply(age_parse,axis=1)

In [225]:
pattern_ymd_dot = re.compile("\s?[0-9]{0,4}\s?\.?\s?[0-9]{1,2}\s?\.\s?[0-9]{1,2}")
pattern_ymd_dash = re.compile("\s?[0-9]{0,4}\s?-?\s?[0-9]{1,2}\s?-\s?[0-9]{1,2}")
pattern_ymd_korean = re.compile(r"\s?[0-9]{0,4}년?\s?[0-9]{1,2}\s?월\s?[0-9]{0,2}일?")
pattern_d_wave = re.compile(r'[~∼～]')
pattern_buter = re.compile(r"부터")
pattern_kkagi = re.compile(r"까지")
pattern_end_num = re.compile(r"[0-9]{1,2}\s?$")
pattern_hakgi = re.compile(r"[0-9]{1,2}\.[0-9]{1,2}학기")
pattern_ymd_korean_wild_card = re.compile(r"\s?[0-9]{0,4}\s?년?\s?[0-9]{0,2}\s?월?\s?[0-9]{0,2}일?") ##for speicfic case
pattern_d_korean = re.compile(r"^\s?[0-9]{1,2}일")
pattern_refine = re.compile(r"([0-9]\.[0-9])|([0-9]일)|([0-9]월)")
pattern_nums = re.compile(r"[0-9]+")

In [226]:
df['split_date'] = df['신청기간'].apply(lambda x: pattern_d_wave.split(x))

In [227]:
def find_date_type(string,flag=1):
    ## flag if 0 then not use wild card 
    
    x = pattern_hakgi.sub("",string)
    
    temp = []
    res = []
    
    if pattern_d_korean.search(x):
        res.extend(pattern_d_korean.findall(x))
    
    if pattern_ymd_dot.search(x):
        res.extend(pattern_ymd_dot.findall(x))
    elif pattern_ymd_dash.search(x):
        res.extend(pattern_ymd_dash.findall(x))
    elif pattern_ymd_korean.search(x):
        res.extend(pattern_ymd_korean.findall(x))
    elif pattern_ymd_korean_wild_card.search(x) and flag!=0:
        res.extend(pattern_ymd_korean_wild_card.findall(x))
        res=list(set(res))
        try:
            res.remove('')
        except Exception:
            pass
    
    if pattern_end_num.search(x):
        temp = pattern_end_num.findall(x)
    
        if pattern_end_num.search(res[-1]):
            temp2 = pattern_end_num.findall(res[-1])
            
            ## useless code?
            if temp[-1] == temp2[-1] or temp[-1] == temp2[-1]+" ":
                pass
            else:
                res.extend(temp)
                
            ## untill this
        else:
            res.extend(temp)
    
    return res
        

In [228]:
def preprocess_date(parsed_list):
    
    date_pair_list = []
    temp_pair_date = []
    
    if len(parsed_list)<2:
        date_pair_list = find_date_type(parsed_list[0],flag=0)
    else:
        for idx in parsed_list:
            temp_pair_date = find_date_type(idx)
            date_pair_list.append(temp_pair_date)
    
    return date_pair_list

In [229]:
df['date_pair'] = df['split_date'].apply(lambda x :preprocess_date(x) )

In [230]:
def preprocess_date_refine(x):
    
    res_pair = []
    temp_pair = []
    
    if len(x) == 0 :
        return []
    elif type(x[0]) == list:
        n = 0
        for idx in x:
            if len(idx) == 2:
                if pattern_refine.search(idx[0]):
                    temp_pair.append(idx[0])
                    res_pair.append(temp_pair)
                    temp_pair = []
                    temp_pair.append(idx[1])
                else:
                    temp_pair.append('')
                    pass
            elif len(idx) == 1:
                temp_pair.append(idx[0])
            n = n+1
    elif type(x[0]) == str:
        for idx in x:
            res_pair.append([idx])
    
    if temp_pair!=[]:
        res_pair.append(temp_pair)
    
    return res_pair

In [231]:
df['date_refine'] = df['date_pair'].apply(lambda x:preprocess_date_refine(x))

In [232]:
df['date_buter']=df['신청기간'].apply(lambda x: True if pattern_buter.search(x) else False)

In [233]:
df['date_ggagi']=df['신청기간'].apply(lambda x: True if pattern_kkagi.search(x) else False)

In [234]:
pattern_y = re.compile(r"[0-9]{0,4}(?=년)")
pattern_m = re.compile(r"[0-9]{0,2}(?=월)")
pattern_d = re.compile(r"[0-9]{0,2}(?=일)")
pattern_not_num = re.compile(r"[^0-9]")


In [235]:
def refine_string_to_date_time(x,flag,year=None,month=None,day=None):
    # x is string
    # flag 0 startdate 1 enddate
    
    y=year
    m=month
    d=day
    
    Now = datetime.datetime.now()
    
    split_list = []
    temp_list = []
    
    split_list = x.split(" ")
    for idx in split_list:
        temp_list.extend(idx.split("."))
    
    split_list = temp_list
    
    for idx in split_list:

        if idx =='':
            pass
        elif pattern_not_num.search(idx):
            if pattern_y.search(x):
                y = pattern_y.search(x).group()
            if pattern_m.search(x):
                m = pattern_m.search(x).group()
            if pattern_d.search(x):
                d = pattern_d.search(x).group()
        else:
            temp = int(idx)
            if temp == 0:
                pass
            elif temp > 12 and y == None:
                y = temp
            elif m==None and temp<13:
                m = temp
            elif m !=None and d==None:
                d=temp
    
    
    if y == None or y == '':
        y = Now.year
    else:
        if int(y) < 100:
            y = int(y) + 2000
    
    if m ==None or m =='':
        m=Now.month
    if d ==None or d =='':
        if flag ==0:
            d = 1
        elif flag ==1:
            d=calendar.monthrange(int(y),int(m))[1]
        
    return datetime.datetime(int(y),int(m),int(d))

In [236]:
def refine_to_datetime(df_in):
    df = df_in.copy()
    x=df['date_refine']
    n1 = 0
    n2 = 0
    res_list = []
    start_date = None
    end_date = None
    
    if len(x) == 0:
        res_list = [] 
    else:
        for idx in x:
            start_date = None
            end_date = None

            if len(idx)==2:
                n1 = len(pattern_nums.findall(idx[0]))
                n2 = len(pattern_nums.findall(idx[1]))
                
                if n1 == 0 and n2 == 0:
                    pass
                elif n1 == 0:
                    start_date =None
                    end_date = refine_string_to_date_time(idx[1],0)
                elif n2 ==0:
                    start_date = refine_string_to_date_time(idx[0],1)
                    end_date = None
                else:
                    if n1==n2:
                        start_date = refine_string_to_date_time(idx[0],0)
                        end_date = refine_string_to_date_time(idx[1],1)
                    elif n1>n2:
                        if n1==3:
                            start_date = refine_string_to_date_time(idx[0],0)
                            end_date = refine_string_to_date_time(idx[1],1,year = start_date.year)
                        elif n1==2:
                            
                            start_date = refine_string_to_date_time(idx[0],0)
                            end_date = start_date = refine_string_to_date_time(idx[1],1,month= start_date.month)

                    elif n2>n1:
                        if n2==3:
                            end_date  = refine_string_to_date_time(idx[1],1)
                            start_date = refine_string_to_date_time(idx[0],0,year = end_date.year)
                        elif n2 ==2:
                            end_date = refine_string_to_date_time(idx[1],1)
                            start_date = refine_string_to_date_time(idx[0],0,month = end_date.month)
            elif len(idx) == 1:
                if df['date_buter']==True:
                    start_date = refine_string_to_date_time(idx[0],0)
                elif df['date_ggagi']==True:
                    end_date  = refine_string_to_date_time(idx[0],1)
                else:
                    start_date = refine_string_to_date_time(idx[0],0)
                    end_date = refine_string_to_date_time(idx[0],1)
            elif len(idx) ==0:
                pass
            res_list.append([start_date,end_date])
            
                    
    df['res_date'] = res_list
    
    return df

In [237]:
df['res_date'] = None

In [238]:
df = df.apply(refine_to_datetime, axis = 1)

In [239]:
df.columns

Index(['Unnamed: 0', 'id', 'title', 'hash tag', '설명', '연령', '학력', '전공', '취업상태',
       '특화분야', '기업', '신청기간', '기타내용', 'start_age', 'end_age', 'split_date',
       'date_pair', 'date_refine', 'date_buter', 'date_ggagi', 'res_date'],
      dtype='object')

In [240]:
url_id_base = "https://www.youthcenter.go.kr/jynEmpSptNew/jynEmpSptGuide.do?bizId="

In [241]:
##df = df[['id','title','hash tag','설명','학력','전공','취업상태','start_age','end_age','res_date']]

In [242]:
def date_extract(source_df):
    df = pd.DataFrame(columns = ['id','date'])
    n = 0
    for idx,row in source_df.iterrows():
        if len(row['res_date'])>1:
            df=df.append({'id':row['id'],'date':row['res_date']},ignore_index=True)
            n = n + 1
    return df

In [243]:
df_date = date_extract(df)

In [244]:
def date_list_parse(df_in):
    ## df_in == row of df
    df = df_in.copy()
    df['start_date'] = None
    df['end_date'] = None
    
    if len(df['res_date'])==0:
        pass
    else:
        df['start_date'] = df['res_date'][0][0]
        df['end_date'] = df['res_date'][0][1]
        
    return df

In [245]:
df = df.apply(date_list_parse,axis=1)

In [246]:
df['uri']=df['id'].apply(lambda x: url_id_base+str(x))

In [292]:
pd.set_option("display.max_rows", 500)

## result

In [293]:
df.loc[(df.start_age.notnull()) | (df.end_age.notnull()),['연령','start_age','end_age']]

,연령,start_age,end_age
1,"만 19세 이상(단, 남자는 군필이거나 면제)",19,None
2,만 18~34세,18,34
3,만 18세 이상 ~ 34세 이하,18,34
4,만 15세 이상,15,None
6,만 19세 이상~만 34세 이하의 청년,19,34
7,만 34세 이하,None,34
8,만 15세 이상 34세 이하인 청년,15,34
9,만 39세 이하,None,39
10,만 39세 이하,None,39
11,만 20세 이상~ 만 39세 미만,20,39


In [295]:
df.loc[(df.end_date.notnull()) | (df.start_date.notnull()),['신청기간','start_date','end_date']]

,신청기간,start_date,end_date
2,매월 1일~20일,2019-11-01 00:00:00,2019-11-20 00:00:00
5,"상시(지원기간: 2018. 7월부터, 예산 소진 시 조기마감 주의)",2018-07-01 00:00:00,None
6,2018년 7월 31일~2021년 12월 31일,2018-07-31 00:00:00,2021-12-31 00:00:00
10,"매년 1~2월, 6~7월 신청 공고 및 접수(연 2회)",2019-01-01 00:00:00,2019-02-28 00:00:00
16,(`19년 기준) 모집 일정 1차: `19.01.28. ~ `19.02.15. ...,2019-01-28 00:00:00,2019-02-15 00:00:00
17,상시 (~ 2021. 12. 31.까지 일몰기한)* 일몰기한 : 세금을 감면해주는 기한,None,2021-12-31 00:00:00
21,‘19년 9월(예정)부터 신청자가 온라인 신청 및 서류 제출,2019-09-01 00:00:00,None
27,[홍제동행복(연합)기숙사] 2019.01.22(화)~2019.01.27(일)[부산행...,2019-01-22 00:00:00,2019-01-27 00:00:00
39,2018년 8월 중 예정,2018-08-01 00:00:00,2018-08-31 00:00:00
44,7월 ~ 8월(지자체 별 상이),2019-07-01 00:00:00,2019-08-31 00:00:00
